# 5555

## What's new:

1- Fully Vectorized NumPy Implementation


In [2]:
import numpy as np
import pandas as pd

np.random.seed(42)  # reproducibility

In [4]:
predicted_df = pd.DataFrame({
    "DATETIME": pd.date_range("2025-08-13 22:00:00", periods=10, freq="h"),
    "forecast_class": np.random.choice([0, 1, 2], size=10),
    "prob_0": np.random.rand(10),
    "prob_1": np.random.rand(10),
    "prob_2": np.random.rand(10),
})


vals = predicted_df["forecast_class"].tolist()
print("[" + " ".join(str(x) for x in vals) + "]")
def filter_forecast_classes_vectorized(fc):
    fc = np.array(fc, dtype=int)

    # ---------------------------------------------------------
    # RULE A: Remove consecutive duplicates, keep only the last
    # ---------------------------------------------------------
    # Find positions where the next value is identical
    same_next = np.r_[False, fc[1:] == fc[:-1]]

    # Zero out all but the last element of consecutive blocks
    maskA = same_next & np.isin(fc, [1, 2])
    fc[maskA] = 0

    # ---------------------------------------------------------
    # RULE B & C: If val appears, then zeros, then val → keep only newest
    # ---------------------------------------------------------
    for val in (1, 2):
        positions = np.where(fc == val)[0]

        if len(positions) > 1:
            # Zero out all but the last occurrence of this class
            fc[positions[:-1]] = 0

    return fc.tolist()

filtered = filter_forecast_classes_vectorized(predicted_df["forecast_class"].tolist())
print("[" + " ".join(str(x) for x in filtered) + "]")

[0 1 0 1 2 2 0 2 2 1]
[0 0 0 0 0 0 0 2 0 1]


In [6]:
filtered = filter_forecast_classes_vectorized(predicted_df["forecast_class"].to_numpy())
predicted_df["forecast_class"] = filtered

predicted_df

,DATETIME,forecast_class,prob_0,prob_1,prob_2
0,2025-08-13 22:00:00,0,0.391061,0.926659,0.965255
1,2025-08-13 23:00:00,0,0.182236,0.727272,0.607034
2,2025-08-14 00:00:00,0,0.755361,0.326541,0.275999
3,2025-08-14 01:00:00,0,0.425156,0.570444,0.296274
4,2025-08-14 02:00:00,0,0.207942,0.520834,0.165267
5,2025-08-14 03:00:00,0,0.567700,0.961172,0.015636
6,2025-08-14 04:00:00,0,0.031313,0.844534,0.423401
7,2025-08-14 05:00:00,2,0.842285,0.747320,0.394882
8,2025-08-14 06:00:00,0,0.449754,0.539692,0.293488
9,2025-08-14 07:00:00,1,0.395150,0.586751,0.014080
